# The SWAP Test Algorithm

The swap test is a quantum function that checks the overlap between two quantum states: the inputs of the function are two quantum registers of the same size,$\Ket{\psi_1 ,\Ket{\psi_2}$, and it returns as an output a single "test" qubit whose state encodes the overlap between the two inputs:  $\Ket{q}_{test} = \alpha\Ket{0}+\sqrt{1-\alpha^2}\Ket{1}$, with $$ \alpha^2 = \frac{1}{2}\left(1+|\langle \psi_1 |\psi_2 \rangle |^2\right). $$ Thus, the probability of measuring the test qubit at state $\Ket{0}$ is 1, if the states are identical (up to a global phase) and 0.5 if the states are orthogonal to each other.

In summary, the SWAP test in quantum computing allows for efficient and precise comparison of quantum states, enabling applications in quantum machine learning, quantum cryptography, and beyond.

![Swap test circuit](./images/swap_test_circuit.png)

The quantum model starts with an H gate on the test qubit, followed by swapping between the two states controlled on the test qubit (a controlled-SWAP gate for each of the qubits in the two states), and a final H gate on the test qubit:

In [1]:
#### Prepare two random states:
import numpy as np

np.random.seed(12)

NUM_QUBITS = 3
amps1 = 1 - 2 * np.random.rand(
    2**NUM_QUBITS
)  # vector of 2^3 numbers in the range [-1,1]
amps2 = 1 - 2 * np.random.rand(2**NUM_QUBITS)
amps1 = amps1 / np.linalg.norm(amps1)  # normalize the vector
amps2 = amps2 / np.linalg.norm(amps2)

In [45]:
amps1

numpy.ndarray

[pepare_amplitudes](https://github.com/Classiq/classiq-library/blob/main/functions/qmod_library_reference/qmod_core_library/prepare_state_and_amplitudes/prepare_state_and_amplitudes.ipynb)
Create a model and synthesize:

In [12]:
from classiq import (
    Output,
    QArray,
    QBit,
    create_model,
    execute,
    prepare_amplitudes,
    qfunc,
    set_execution_preferences,
    swap_test,
    synthesize,
)
from classiq.execution import ExecutionPreferences


@qfunc
def main(test: Output[QBit]):

    state1 = QArray("state1")
    state2 = QArray("state2")
    prepare_amplitudes(amps1.tolist(), 0.0,state1)
    prepare_amplitudes(amps2.tolist(), 0.0,state2)
    swap_test(state1, state2, test)


qmod = create_model(main)
qmod = set_execution_preferences(
    qmod, execution_preferences=ExecutionPreferences(num_shots=100_000)
)
qprog = synthesize(qmod)

ehj


[SWAT Test definition](https://github.com/Classiq/classiq-library/blob/main/functions/open_library_definitions/swap_test.qmod)

In [4]:
from classiq import show

show(qprog)

Opening: https://platform.classiq.io/circuit/7c71a6d2-660a-42e5-8813-21980dcd7aab?version=0.43.0


In [5]:
res = execute(qprog).result()

In [6]:
res

[TaggedExecutionDetails(value_type=<SavedResultValueType.ExecutionDetails: 'ExecutionDetails'>, name='result', value=ExecutionDetails(vendor_format_result={}, counts={'0': 61175, '1': 38825}, counts_lsb_right=True, parsed_states={'0': {'test': 0.0}, '1': {'test': 1.0}}, histogram=None, output_qubits_map={'test': (0,)}, state_vector=None, parsed_state_vector_states=None, physical_qubits_map={'test': (6,)}, num_shots=100000))]

Using the expected probability of measuring the state
 as defined above: $$ \alpha^2 = \frac{1}{2}\left(1+|\langle \psi_1 |\psi_2 \rangle |^2\right). $$ we extract the overlap
$$|\Braket{\psi_1 | \psi_2}|^2 = \sqrt{2P(q_{test} = \Bra{0})-1}$$
The exact overlap is computed with the dot product of the two state-vectors. Note that for the sake of this demonstration we execute this circuit
 times to improve the precision of the probability estimate. This is usually not required in actual programs.

In [7]:
overlap_from_swap_test = np.sqrt(
    2 * res[0].value.counts["0"] / sum(res[0].value.counts.values()) - 1
)
exact_overlap = np.abs(amps1 @ amps2)

In [8]:
print("States overlap from Swap-Test result:", overlap_from_swap_test)
print("States overlap from classical calculation:", exact_overlap)

States overlap from Swap-Test result: 0.47275786614291254
States overlap from classical calculation: 0.46972037234759095


In [9]:
RTOL = 0.05
assert np.isclose(
    overlap_from_swap_test, exact_overlap, RTOL
), f"""
The quantum result is too far than classical one, by a relative tolerance of {RTOL}. Please verify your parameters"""

# Task 1
Use the pre-built example of the SWAP Test Links to an external site.from the IDE, and adapt the two states to be 2-qubit states with 0 overlap (i.e. orthogonal states). You can use the prepare_stateLinks to an external site. function for this (use a standard error bound of 0.01). Synthesize and execute the algorithm using Classiq’s simulator. What was the probability of measuring 0 in the test qubit? Is it as you expected?

[State Preparation Classiq](https://github.com/Classiq/classiq-library/blob/main/functions/qmod_library_reference/qmod_core_library/prepare_state_and_amplitudes/prepare_state_and_amplitudes.ipynb)

In [33]:
from classiq import prepare_state
@qfunc
def main(test: Output[QBit]):
    state1 = QArray("state1")
    state2 = QArray("state2")
    prepare_state(probabilities=[1,0,0,0], bound=0.01, out=state1)
    prepare_state(probabilities=[0,0,0,1], bound=0.01, out=state2)
    swap_test(state1,state2,test)

qmod = create_model(main)
qmod = set_execution_preferences(
    qmod, execution_preferences=ExecutionPreferences(num_shots=100_000)
)

In [34]:
from classiq import synthesize

qprog = synthesize(qmod)

In [35]:
res = execute(qprog).result()

In [36]:
overlap_from_swap_test = np.sqrt(
    2 * res[0].value.counts["0"] / sum(res[0].value.counts.values()) - 1
)

In [37]:
print("States overlap from Swap-Test result:", overlap_from_swap_test)

States overlap from Swap-Test result: 0.05215361924162172


## Task 2:
Repeat Task 1 but now adapt the two states to be 2-qubit states with perfect overlap (i.e. to be identical states).

In [ ]:
prepare_state(probabilities=[1,0,0,0], bound=0.01, out=state1)
prepare_state(probabilities=[1,0,0,0], bound=0.01, out=state2)

# Task 3
Repeat Task 1 but now adapt the two states to be 2-qubit states with 0.5 fidelity overlap (i.e. $|\Braket{\psi_1 | \psi_2}|^2 = 0.5$)

$\Ket{\psi_1} = \frac{\Ket{00} + \Ket{11}}{\sqrt{2}}$
$\Ket{\psi_2} = \frac{\Ket{00} - \Ket{11}}{\sqrt{2}}$

$$ |\Braket{\psi_1 | \psi_2}| = \frac{1}{\sqrt{2}}$$

In [50]:
amps1 = np.array([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)])
amps2 = np.array([1/np.sqrt(2), 0, 0, -1/np.sqrt(2)])

array([0.70710678, 0.        , 0.        , 0.70710678])

In [51]:
from classiq import prepare_state
@qfunc
def main(test: Output[QBit]):
    state1 = QArray("state1")
    state2 = QArray("state2")
    prepare_amplitudes(amps1.tolist(), 0.0,state1)
    prepare_amplitudes(amps2.tolist(), 0.0,state2)
    swap_test(state1,state2,test)

qmod = create_model(main)
qmod = set_execution_preferences(
    qmod, execution_preferences=ExecutionPreferences(num_shots=100_000)
)

In [56]:
from classiq import synthesize

qprog = synthesize(qmod)
res = execute(qprog).result()
res[0].value.counts["0"]

49963

In [57]:
sum(res[0].value.counts.values())

100000

In [59]:
res[0].value.counts["0"]/sum(res[0].value.counts.values())

0.49963

In [61]:

overlap_from_swap_test = 2 * res[0].value.counts["0"] / sum(res[0].value.counts.values()) - 1


In [62]:
print("States overlap from Swap-Test result:", overlap_from_swap_test)

States overlap from Swap-Test result: -0.0007399999999999629


# Task 4
Follow the implementation in class and code in the IDE the Hadamard test for calculating the (real part of the) expectation value of the QFT unitary at the state |0> with 4 qubits.  Execute the algorithm from the IDE with 1,000 shots, and calculate the (real part of the) expectation value using the formula
. Is it what you expected it to be?